# 🌍 Somaliland District-Level Drought Prediction Model

**Project:** Drought Early Warning System — Somaliland  
**Author:** Climate Data Science Team  
**Version:** 2.0.0  

---

## Objectives

Build a **district-level drought prediction model** that forecasts drought conditions **3 months ahead** using multi-source climate and socioeconomic data.

| Target | Type | Description |
|--------|------|-------------|
| `target_spi3` | Regression | SPI-3 value at t+3 months |
| `target_drought` | Classification | 1 if SPI-3 < -1 at t+3 |

## Data Sources

| Source | Variables | Resolution |
|--------|-----------|------------|
| **CHIRPS v2** | Monthly rainfall | 0.05° / monthly |
| **ERA5-Land** | Temperature, soil moisture, PET | 0.1° / monthly |
| **FAO SWALIM** | Station rainfall, ASIS index | District / monthly |
| **World Bank** | GDP, rural pop, agriculture | National / annual |
| **NOAA CPC** | ONI/ENSO index | Global / monthly |

## Somaliland Climate Context

Somaliland has two rainy seasons:
- **Gu** (April–June): Long rains — primary agricultural season (~60% of annual rainfall)
- **Deyr** (October–November): Short rains — secondary season (~25% of annual rainfall)
- Dry seasons: **Hagaa** (Jul–Sep) and **Jilaal** (Dec–Mar)

Drought is driven by: ENSO variability, Indian Ocean Dipole, land degradation, and warming trends.

---

## 1. Environment Setup

In [ ]:
# ============================================================
# Install required packages (run once)
# ============================================================
import subprocess, sys

packages = [
    'pandas', 'numpy', 'matplotlib', 'seaborn', 'scikit-learn',
    'xgboost', 'lightgbm', 'shap', 'scipy', 'statsmodels',
    'requests', 'geopandas', 'shapely', 'joblib', 'plotly', 'tqdm', 'wbgapi'
]
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
print('✅ All packages installed.')

In [ ]:
# ============================================================
# Imports
# ============================================================
import sys
sys.path.insert(0, '../src')

import os
import logging
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd
from shapely.geometry import Point

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    mean_squared_error, r2_score, mean_absolute_error,
    roc_auc_score, classification_report, confusion_matrix,
    ConfusionMatrixDisplay, precision_recall_fscore_support
)

import xgboost as xgb
import lightgbm as lgb
import shap
import joblib
import requests
from tqdm import tqdm

warnings.filterwarnings('ignore')

# ── Logging ─────────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s'
)
logger = logging.getLogger(__name__)

# ── Reproducibility ─────────────────────────────────────────
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ── Plot style ───────────────────────────────────────────────
plt.rcParams.update({
    'figure.dpi': 120,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3
})
sns.set_palette('husl')

# ── Paths ────────────────────────────────────────────────────
BASE_DIR    = Path('..').resolve()
DATA_DIR    = BASE_DIR / 'data'
RAW_DIR     = DATA_DIR / 'raw'
PROC_DIR    = DATA_DIR / 'processed'
MODELS_DIR  = BASE_DIR / 'models'
FIGURES_DIR = BASE_DIR / 'reports' / 'figures'

for d in [RAW_DIR, PROC_DIR, MODELS_DIR, FIGURES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ── Somaliland constants ─────────────────────────────────────
BBOX = dict(min_lon=42.5, max_lon=49.5, min_lat=8.0, max_lat=11.5)

DISTRICTS = [
    'Hargeisa', 'Berbera', 'Borama', 'Burao', 'Erigavo',
    'Las_Anod', 'Gabiley', 'Sheikh', 'Odweyne', 'Zeila'
]

DISTRICT_COORDS = {
    'Hargeisa': (44.065, 9.560), 'Berbera':  (45.014, 10.439),
    'Borama':   (43.183, 9.935), 'Burao':    (45.533,  9.517),
    'Erigavo':  (47.367, 10.617),'Las_Anod': (47.367,  8.483),
    'Gabiley':  (43.467, 9.983), 'Sheikh':   (45.200,  9.933),
    'Odweyne':  (45.067, 9.417), 'Zeila':    (43.483, 11.350)
}

FORECAST_HORIZON  = 3
DROUGHT_THRESHOLD = -1.0
TRAIN_END   = '2015-12-31'
TEST_START  = '2016-01-01'

DATE_RANGE = pd.date_range('1981-01', '2023-12', freq='MS')

print('✅ Environment configured.')
print(f'   Base dir : {BASE_DIR}')
print(f'   Date range: {DATE_RANGE[0].date()} → {DATE_RANGE[-1].date()}')

---
## 2. Data Ingestion

We pull from five sources. Each has a live API attempt with a graceful fallback to realistic synthetic data (preserving correct climatological statistics) so the notebook always runs end-to-end.

In [ ]:
# ── src/data_loader.py functions inlined for notebook self-containment ──

def safe_get(url, timeout=20, **kwargs):
    """HTTP GET with timeout and error handling."""
    try:
        r = requests.get(url, timeout=timeout, **kwargs)
        r.raise_for_status()
        return r
    except Exception as e:
        logger.warning(f'HTTP failed {url}: {e}')
        return None


# ────────────────────────────────────────────────────────────
# 2A. CHIRPS — Climate Hazards Group InfraRed Precipitation
#     URL: https://data.chc.ucsb.edu/products/CHIRPS-2.0/
#     Resolution: 0.05°, 1981-present
#     Production: download monthly GeoTIFFs → zonal stats per district
# ────────────────────────────────────────────────────────────
def fetch_chirps(districts, date_range, seed=RANDOM_SEED):
    """
    Fetch CHIRPS monthly rainfall.
    Production: download GeoTIFFs and compute zonal statistics.
    Fallback: synthetic data with correct bimodal Somaliland climatology.
    """
    # ---- Production download template ----
    # for year in range(1981, 2024):
    #     for month in range(1, 13):
    #         fn = f'chirps-v2.0.{year}.{month:02d}.tif.gz'
    #         url = f'https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_monthly/tifs/{fn}'
    #         safe_download(url, RAW_DIR / fn)
    # then: rasterstats.zonal_stats(district_gdf, tif, stats=['mean'])

    logger.info('Generating CHIRPS-like synthetic rainfall ...')
    rng = np.random.default_rng(seed)
    # Monthly climatology (mm) — bimodal Somaliland pattern
    #            J    F    M    A    M    J    J    A    S    O    N    D
    clim = np.array([5,   8,  18,  55,  90,  60,  12,   8,   5,  45,  60,  10], dtype=float)
    mult = dict(Hargeisa=1.0, Berbera=0.4, Borama=1.1, Burao=1.2,
                Erigavo=1.5, Las_Anod=1.3, Gabiley=1.1,
                Sheikh=1.2, Odweyne=1.1, Zeila=0.6)
    rows = []
    for d in districts:
        m_mult = mult.get(d, 1.0)
        for dt in date_range:
            mean_r = clim[dt.month - 1] * m_mult
            rain = max(0, rng.gamma(1.5, mean_r / 1.5) - 0.04 * (dt.year - 1981))
            rows.append({'date': dt, 'district': d, 'rainfall_mm': round(rain, 2)})
    return pd.DataFrame(rows)


# ────────────────────────────────────────────────────────────
# 2B. ERA5-Land — ECMWF Reanalysis v5 Land
#     Access via CDS API: https://cds.climate.copernicus.eu/
#     Requires: ~/.cdsapirc with UID and API key
# ────────────────────────────────────────────────────────────
def fetch_era5(districts, date_range, seed=RANDOM_SEED):
    """
    Fetch ERA5-Land monthly climate variables.
    Production: use cdsapi.Client() to download NetCDF then xarray for extraction.
    Fallback: synthetic ERA5-like data.
    """
    # ---- CDS API template ----
    # import cdsapi
    # c = cdsapi.Client()
    # c.retrieve('reanalysis-era5-land-monthly-means', {
    #     'product_type': 'monthly_averaged_reanalysis',
    #     'variable': ['2m_temperature', 'volumetric_soil_water_layer_1', 'potential_evaporation'],
    #     'year': [str(y) for y in range(1981, 2024)],
    #     'month': [f'{m:02d}' for m in range(1, 13)],
    #     'time': '00:00',
    #     'area': [11.5, 42.5, 8.0, 49.5],  # N,W,S,E
    #     'format': 'netcdf'
    # }, str(RAW_DIR / 'era5_somaliland.nc'))
    # import xarray as xr
    # ds = xr.open_dataset(RAW_DIR / 'era5_somaliland.nc')

    logger.info('Generating ERA5-like synthetic climate data ...')
    rng = np.random.default_rng(seed + 1)
    #            J    F    M    A    M    J    J    A    S    O    N    D
    t_clim = np.array([23, 24, 26, 27, 28, 28, 25, 25, 26, 26, 25, 23], dtype=float)
    sm_clim= np.array([.12,.11,.12,.16,.22,.20,.16,.14,.13,.17,.20,.13], dtype=float)
    pt_clim= np.array([110,115,130,135,140,135,120,120,125,120,110,105], dtype=float)
    t_off  = dict(Hargeisa=0,Berbera=6,Borama=-1,Burao=1,Erigavo=-2,
                  Las_Anod=3,Gabiley=-1,Sheikh=1,Odweyne=1,Zeila=7)
    rows = []
    for d in districts:
        for dt in date_range:
            m = dt.month - 1
            yr = dt.year - 1981
            rows.append({
                'date': dt, 'district': d,
                'temp_2m_c':  round(t_clim[m] + t_off.get(d,0) + 0.02*yr + rng.normal(0,.8), 2),
                'soil_moisture': round(max(0.05, sm_clim[m] + rng.normal(0,.02)), 4),
                'pet_mm':     round(max(0, pt_clim[m] + 0.3*yr + rng.normal(0,8)), 2)
            })
    return pd.DataFrame(rows)


# ────────────────────────────────────────────────────────────
# 2C. World Bank Open Data via wbgapi
#     Country: SO (Somalia — Somaliland not separately recognised)
# ────────────────────────────────────────────────────────────
def fetch_world_bank(date_range, seed=RANDOM_SEED):
    """Fetch World Bank annual socioeconomic indicators."""
    indicators = {
        'AG.AGR.TRVL.ZS': 'agr_value_added_pct',
        'SP.RUR.TOTL.ZS':  'rural_pop_pct',
        'NY.GDP.PCAP.CD':  'gdp_per_capita',
        'AG.LND.ARBL.ZS':  'arable_land_pct',
    }
    try:
        import wbgapi as wb
        frames = []
        for code, name in indicators.items():
            df = wb.data.DataFrame(code, 'SO', time=range(1981, 2024)).T
            df.index = pd.to_datetime(df.index.str.replace('YR',''), format='%Y')
            df.columns = [name]
            frames.append(df)
        annual = pd.concat(frames, axis=1)
        logger.info('World Bank data fetched via wbgapi.')
        return annual
    except Exception as e:
        logger.warning(f'World Bank API failed: {e}. Using synthetic.')
        rng = np.random.default_rng(seed+2)
        years = sorted(set(date_range.year))
        rows = [{'year': y, 'agr_value_added_pct': 60+rng.normal(0,3)-0.1*i,
                 'rural_pop_pct': 72+rng.normal(0,1)-0.15*i,
                 'gdp_per_capita': 120+3*i+rng.normal(0,10),
                 'arable_land_pct': 1.8+rng.normal(0,.1)}
                for i,y in enumerate(years)]
        return pd.DataFrame(rows).set_index('year')


# ────────────────────────────────────────────────────────────
# 2D. SWALIM / FAO ASIS
#     SWALIM: http://www.faoswalim.org/
#     ASIS:   https://www.fao.org/giews/earthobservation/
#     Both require manual portal download → place CSV in data/raw/
# ────────────────────────────────────────────────────────────
def fetch_swalim(districts, date_range, seed=RANDOM_SEED):
    """Load SWALIM station data + FAO ASIS. Falls back to synthetic."""
    csv_path = RAW_DIR / 'swalim_asis.csv'
    if csv_path.exists():
        df = pd.read_csv(csv_path, parse_dates=['date'])
        logger.info(f'SWALIM data loaded from {csv_path}')
        return df
    logger.warning('SWALIM CSV not found. Using synthetic ASIS.')
    rng = np.random.default_rng(seed+3)
    # ASIS 0–100; higher = more crop stress; peaks in dry months
    asis_clim = np.array([70,65,55,30,20,25,50,55,60,30,25,65], dtype=float)
    rows = []
    for d in districts:
        for dt in date_range:
            rows.append({
                'date': dt, 'district': d,
                'asis_index': round(np.clip(asis_clim[dt.month-1]+rng.normal(0,12),0,100),1)
            })
    return pd.DataFrame(rows)


# ────────────────────────────────────────────────────────────
# 2E. ENSO — NOAA ONI (Oceanic Niño Index)
#     ENSO strongly controls Somaliland rainfall:
#     La Niña → drought risk↑; El Niño → Deyr rains↑
# ────────────────────────────────────────────────────────────
def fetch_oni(date_range, seed=RANDOM_SEED):
    """Download ONI index from NOAA CPC. Fallback: synthetic ENSO."""
    url = 'https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt'
    r = safe_get(url)
    if r is not None:
        try:
            s2m = {'DJF':1,'JFM':2,'FMA':3,'MAM':4,'AMJ':5,'MJJ':6,
                   'JJA':7,'JAS':8,'ASO':9,'SON':10,'OND':11,'NDJ':12}
            rows = []
            for line in r.text.strip().split('\n')[1:]:
                p = line.split()
                if len(p) >= 3:
                    try: rows.append({'date': pd.Timestamp(year=int(p[1]), month=s2m[p[0]], day=1), 'oni': float(p[2])})
                    except: pass
            df = pd.DataFrame(rows).drop_duplicates('date').set_index('date')
            df = df.reindex(date_range).interpolate().reset_index()
            df.columns = ['date','oni']
            logger.info('ONI fetched from NOAA.')
            return df
        except Exception as e:
            logger.warning(f'ONI parse failed: {e}')
    rng = np.random.default_rng(seed+4)
    t = np.arange(len(date_range))
    oni = 0.8*np.sin(2*np.pi*t/48) + 0.4*np.sin(2*np.pi*t/84) + rng.normal(0,.3,len(t))
    return pd.DataFrame({'date': date_range, 'oni': np.round(oni,3)})


# ── Load all sources ─────────────────────────────────────────
chirps_df  = fetch_chirps(DISTRICTS, DATE_RANGE)
era5_df    = fetch_era5(DISTRICTS, DATE_RANGE)
swalim_df  = fetch_swalim(DISTRICTS, DATE_RANGE)
oni_df     = fetch_oni(DATE_RANGE)
wb_df      = fetch_world_bank(DATE_RANGE)

print(f'CHIRPS  : {chirps_df.shape}')
print(f'ERA5    : {era5_df.shape}')
print(f'SWALIM  : {swalim_df.shape}')
print(f'ONI     : {oni_df.shape}')
print(f'WorldBnk: {wb_df.shape}')

---
## 3. Data Cleaning & Merging

In [ ]:
def merge_all(chirps, era5, swalim, oni, wb):
    """Merge all data sources into a single master DataFrame."""
    for df in [chirps, era5, swalim, oni]:
        df['date'] = pd.to_datetime(df['date'])

    master = chirps.merge(era5, on=['date','district'], how='left')
    master = master.merge(swalim, on=['date','district'], how='left')
    master = master.merge(oni, on='date', how='left')

    # World Bank: annual → monthly
    wb_m = wb.copy()
    wb_m.index = pd.to_datetime(wb_m.index, format='%Y')
    all_months = pd.date_range(f'{wb_m.index.min().year}-01',
                               f'{wb_m.index.max().year}-12', freq='MS')
    wb_m = wb_m.reindex(all_months).ffill().bfill()
    wb_m.index.name = 'date'
    master = master.merge(wb_m.reset_index(), on='date', how='left')

    # Calendar features
    master['year']      = master['date'].dt.year
    master['month']     = master['date'].dt.month
    master['month_sin'] = np.sin(2*np.pi*master['month']/12)
    master['month_cos'] = np.cos(2*np.pi*master['month']/12)

    # Somaliland seasons
    def season(m):
        return 'Gu' if m in [4,5,6] else 'Deyr' if m in [10,11] \
               else 'Hagaa' if m in [7,8,9] else 'Jilaal'
    master['season']   = master['month'].apply(season)
    master['is_gu']    = (master['season']=='Gu').astype(int)
    master['is_deyr']  = (master['season']=='Deyr').astype(int)

    # Spatial features
    master['lon'] = master['district'].map(lambda d: DISTRICT_COORDS.get(d,(45,9.5))[0])
    master['lat'] = master['district'].map(lambda d: DISTRICT_COORDS.get(d,(45,9.5))[1])

    # District code
    codes = {d:i for i,d in enumerate(sorted(master['district'].unique()))}
    master['district_code'] = master['district'].map(codes)

    return master.sort_values(['district','date']).reset_index(drop=True)


def clean_data(df):
    """Clean and validate the master dataset."""
    df = df.copy()
    df['rainfall_mm']   = df['rainfall_mm'].clip(lower=0)
    df['soil_moisture'] = df['soil_moisture'].clip(0.0, 0.6)
    df['asis_index']    = df['asis_index'].clip(0, 100)

    # Per-district interpolation (forward-fill up to 2 months, then linear)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols] = (
        df.groupby('district', group_keys=False)[num_cols]
        .apply(lambda g: g.ffill(limit=2).bfill(limit=2).interpolate())
    )
    n0 = len(df)
    df = df.dropna(subset=['rainfall_mm'])
    if len(df) < n0:
        logger.warning(f'Dropped {n0-len(df)} rows with missing rainfall.')
    return df


master_df = merge_all(chirps_df, era5_df, swalim_df, oni_df, wb_df)
master_df = clean_data(master_df)

# Quick quality report
print(f'Master shape : {master_df.shape}')
print(f'Date range   : {master_df["date"].min().date()} → {master_df["date"].max().date()}')
print(f'Districts    : {sorted(master_df["district"].unique())}')
miss = master_df.isnull().sum()
if miss.any(): print('Missing:\n', miss[miss>0])
else: print('✅ No missing values.')

---
## 4. Feature Engineering

In [ ]:
# ── SPI Computation ──────────────────────────────────────────
def compute_spi(rain_series, scale=3, ref_start=1981, ref_end=2010):
    """
    Standardized Precipitation Index via gamma distribution fitting.

    WMO standard method:
    1. Accumulate rainfall over `scale` months
    2. Per calendar month: fit gamma to 1981-2010 reference period
    3. CDF → standard normal quantile = SPI

    SPI classes:
      ≥ +2.0 : Extremely wet
      +1.0 to +1.99 : Moderately wet
      -0.99 to +0.99: Near normal
      -1.0 to -1.49 : Moderate drought
      -1.5 to -1.99 : Severe drought
      ≤ -2.0 : Extreme drought
    """
    rolled = rain_series.rolling(window=scale, min_periods=scale).sum()
    spi = pd.Series(np.nan, index=rain_series.index, name=f'spi_{scale}')

    if not hasattr(rolled.index, 'month'):
        return spi

    ref_mask = (rolled.index.year >= ref_start) & (rolled.index.year <= ref_end)

    for m in range(1, 13):
        m_mask  = rolled.index.month == m
        ref_dat = rolled[m_mask & ref_mask].dropna()
        all_dat = rolled[m_mask].dropna()
        if len(ref_dat) < 8:
            continue
        # Probability of zero rainfall
        q = (ref_dat == 0).mean()
        nz = ref_dat[ref_dat > 0]
        if len(nz) < 5:
            continue
        try:
            shape, _, sc = stats.gamma.fit(nz, floc=0)
            p = (q + (1-q) * stats.gamma.cdf(all_dat, shape, scale=sc)).clip(0.001, 0.999)
            spi[all_dat.index] = stats.norm.ppf(p)
        except Exception:
            pass
    return spi


def add_spi_all_districts(df, scales=(1, 3, 6)):
    """Apply SPI computation across all districts and scales."""
    frames = []
    for scale in scales:
        col = f'spi_{scale}'
        parts = []
        for dist, grp in df.groupby('district'):
            rain = grp.set_index('date')['rainfall_mm'].sort_index()
            s = compute_spi(rain, scale).reset_index()
            s.columns = ['date', col]
            s['district'] = dist
            parts.append(s)
        spi_df = pd.concat(parts, ignore_index=True)
        frames.append(spi_df)
        logger.info(f'SPI-{scale} computed.')
    result = df.copy()
    for sdf in frames:
        col = [c for c in sdf.columns if c.startswith('spi')][0]
        result = result.merge(sdf, on=['date','district'], how='left')
    return result


# ── Anomaly Features ─────────────────────────────────────────
def compute_anomaly(series, month_series, ref_years=(1981,2010)):
    """Z-score anomaly per calendar month relative to reference period."""
    anom = pd.Series(np.nan, index=series.index)
    ref_mask = ((series.index.year >= ref_years[0]) &
                (series.index.year <= ref_years[1]))
    for m in range(1, 13):
        mm = month_series == m
        ref_vals = series[mm & ref_mask]
        all_vals = series[mm]
        if len(ref_vals) > 2 and ref_vals.std() > 0:
            anom[mm] = (all_vals - ref_vals.mean()) / ref_vals.std()
    return anom


def add_anomaly_features(df):
    """Add standardized anomaly features per district."""
    var_map = {
        'rainfall_mm':   'rain_anom',
        'temp_2m_c':     'temp_anom',
        'soil_moisture': 'sm_anom',
        'pet_mm':        'pet_anom',
    }
    df = df.copy()
    result_parts = []
    for dist, grp in df.groupby('district'):
        grp = grp.sort_values('date').copy()
        grp = grp.set_index('date')
        months = grp['month']
        for var, name in var_map.items():
            if var in grp.columns:
                grp[name] = compute_anomaly(grp[var], months)
        result_parts.append(grp.reset_index())
    return pd.concat(result_parts, ignore_index=True)


# ── Lag + Rolling Features ───────────────────────────────────
def add_lag_rolling(df, lag_vars, lags=(1,3,6), windows=(3,6,12)):
    """
    Add lag and rolling mean features per district.
    Computed WITHIN each district to prevent spatial leakage.
    """
    parts = []
    for dist, grp in df.groupby('district'):
        grp = grp.sort_values('date').copy()
        for var in lag_vars:
            if var not in grp.columns: continue
            for lag in lags:
                grp[f'{var}_lag{lag}'] = grp[var].shift(lag)
            for w in windows:
                grp[f'{var}_roll{w}m'] = grp[var].rolling(w, min_periods=max(1,w//2)).mean()
        parts.append(grp)
    return pd.concat(parts, ignore_index=True)


# ── Derived / Interaction Features ───────────────────────────
def add_derived(df):
    """Add physically-motivated composite features."""
    df = df.copy()
    # Aridity: P/PET ratio (low = arid)
    df['aridity_idx']  = df['rainfall_mm'] / (df['pet_mm'].clip(lower=1))
    # Water deficit: PET − P (positive = moisture stressed)
    df['water_deficit'] = (df['pet_mm'] - df['rainfall_mm']).clip(lower=0)
    # Drought stress: high temp × low soil moisture
    df['drought_stress'] = df['temp_anom'] * (-df['sm_anom'].clip(upper=0))
    # ENSO × season interactions
    df['oni_gu']    = df['oni'] * df['is_gu']
    df['oni_deyr']  = df['oni'] * df['is_deyr']
    # Year-on-year rainfall change
    df = df.sort_values(['district','date'])
    df['rain_yoy'] = df.groupby('district')['rainfall_mm'].pct_change(12) * 100
    return df


# ── Apply full pipeline ───────────────────────────────────────
master_df = add_spi_all_districts(master_df)
master_df = add_anomaly_features(master_df)

LAG_VARS = [
    'rainfall_mm','spi_1','spi_3','temp_2m_c',
    'soil_moisture','oni','pet_mm','rain_anom','sm_anom','asis_index'
]
master_df = add_lag_rolling(master_df, LAG_VARS)
master_df = add_derived(master_df)

print(f'✅ Features engineered. Dataset shape: {master_df.shape}')
print(f'   Total columns: {len(master_df.columns)}')

---
## 5. Target Variable Construction

In [ ]:
def create_targets(df, horizon=FORECAST_HORIZON, threshold=DROUGHT_THRESHOLD):
    """
    Create forecast targets.

    CRITICAL: We shift SPI-3 BACKWARDS by `horizon` months so that
    features at time t predict conditions at time t+horizon.
    Shift is done PER DISTRICT to avoid cross-district contamination.
    """
    parts = []
    for dist, grp in df.groupby('district'):
        grp = grp.sort_values('date').copy()
        grp['target_spi3']    = grp['spi_3'].shift(-horizon)
        grp['target_drought'] = (grp['target_spi3'] < threshold).astype(float)
        parts.append(grp)
    result = pd.concat(parts, ignore_index=True)
    n0 = len(result)
    result = result.dropna(subset=['target_spi3'])
    result['target_drought'] = result['target_drought'].astype(int)
    logger.info(f'Dropped {n0-len(result)} rows (future target NaN).')
    logger.info(f'Drought prevalence: {result["target_drought"].mean()*100:.1f}%')
    return result


master_df = create_targets(master_df)

print(f'Target SPI-3 range : [{master_df["target_spi3"].min():.2f}, {master_df["target_spi3"].max():.2f}]')
print(f'Drought prevalence : {master_df["target_drought"].mean()*100:.1f}%')
print(master_df['target_drought'].value_counts().rename({0:'No Drought',1:'Drought'}))

---
## 6. Train / Test Split

In [ ]:
# Feature columns: all numeric except identifiers and targets
EXCLUDE = {'date','district','season','target_spi3','target_drought','spi_3','spi_6'}
FEATURE_COLS = [
    c for c in master_df.select_dtypes(include=[np.number]).columns
    if c not in EXCLUDE and not c.startswith('target_')
]

# Remove rows with NaN features (lag/rolling warm-up)
model_df = master_df.dropna(subset=FEATURE_COLS + ['target_spi3'])
model_df = model_df[model_df['year'] >= 1985]  # 4-year burn-in for lag features

train_df = model_df[model_df['date'] <= TRAIN_END].copy()
test_df  = model_df[model_df['date'] >= TEST_START].copy()

X_train, X_test = train_df[FEATURE_COLS], test_df[FEATURE_COLS]
y_train_reg, y_test_reg = train_df['target_spi3'], test_df['target_spi3']
y_train_cls, y_test_cls = train_df['target_drought'], test_df['target_drought']

print('=' * 55)
print('TEMPORAL TRAIN / TEST SPLIT')
print('=' * 55)
print(f'Train: {train_df["date"].min().date()} → {train_df["date"].max().date()} | {len(train_df):,} samples')
print(f'Test : {test_df["date"].min().date()} → {test_df["date"].max().date()} | {len(test_df):,} samples')
print(f'Features : {len(FEATURE_COLS)}')
print(f'Train drought prevalence: {y_train_cls.mean()*100:.1f}%')
print(f'Test  drought prevalence: {y_test_cls.mean()*100:.1f}%')

---
## 7. Modeling

In [ ]:
# ── Pipeline factory ─────────────────────────────────────────
def make_pipeline(model):
    return Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler',  StandardScaler()),
        ('model',   model)
    ])


def eval_reg(name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    mae  = mean_absolute_error(y_true, y_pred)
    print(f'  {name:<15s} RMSE={rmse:.4f}  R²={r2:.4f}  MAE={mae:.4f}')
    return dict(model=name, rmse=rmse, r2=r2, mae=mae)


def eval_cls(name, y_true, y_pred, y_prob):
    auc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan')
    p,r,f1,_ = precision_recall_fscore_support(y_true,y_pred,average='binary',zero_division=0)
    print(f'  {name:<15s} AUC={auc:.4f}  F1={f1:.4f}  P={p:.4f}  R={r:.4f}')
    return dict(model=name, auc=auc, f1=f1, precision=p, recall=r)


pos_ratio = (y_train_cls==0).sum() / max((y_train_cls==1).sum(), 1)

# ── Regression ───────────────────────────────────────────────
REG_MODELS = {
    'RandomForest': RandomForestRegressor(
        n_estimators=200, max_depth=12, min_samples_leaf=5,
        n_jobs=-1, random_state=RANDOM_SEED),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=300, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1, random_state=RANDOM_SEED, verbosity=0),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=300, max_depth=8, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        n_jobs=-1, random_state=RANDOM_SEED, verbose=-1),
}

# ── Classification ───────────────────────────────────────────
CLS_MODELS = {
    'RandomForest': RandomForestClassifier(
        n_estimators=200, max_depth=12, min_samples_leaf=5,
        class_weight='balanced', n_jobs=-1, random_state=RANDOM_SEED),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=300, max_depth=6, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        scale_pos_weight=pos_ratio,
        n_jobs=-1, random_state=RANDOM_SEED, verbosity=0),
    'LightGBM': lgb.LGBMClassifier(
        n_estimators=300, max_depth=8, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        class_weight='balanced',
        n_jobs=-1, random_state=RANDOM_SEED, verbose=-1),
}

# ── Train & evaluate ─────────────────────────────────────────
print('REGRESSION — SPI-3 (3-month ahead)')
print('-' * 55)
reg_pipes, reg_res, reg_preds = {}, [], {}
for name, mdl in REG_MODELS.items():
    pipe = make_pipeline(mdl)
    pipe.fit(X_train, y_train_reg)
    pred = pipe.predict(X_test)
    reg_pipes[name] = pipe
    reg_preds[name] = pred
    reg_res.append(eval_reg(name, y_test_reg, pred))

print()
print('CLASSIFICATION — Drought binary (3-month ahead)')
print('-' * 55)
cls_pipes, cls_res, cls_preds, cls_probs = {}, [], {}, {}
for name, mdl in CLS_MODELS.items():
    pipe = make_pipeline(mdl)
    pipe.fit(X_train, y_train_cls)
    pred = pipe.predict(X_test)
    prob = pipe.predict_proba(X_test)[:,1]
    cls_pipes[name] = pipe
    cls_preds[name] = pred
    cls_probs[name] = prob
    cls_res.append(eval_cls(name, y_test_cls, pred, prob))

reg_df = pd.DataFrame(reg_res)
cls_df = pd.DataFrame(cls_res)
best_reg = reg_df.loc[reg_df['rmse'].idxmin(), 'model']
best_cls = cls_df.loc[cls_df['f1'].idxmax(), 'model']
print(f'\n🏆 Best regression model    : {best_reg}')
print(f'🏆 Best classification model: {best_cls}')

In [ ]:
# ── Confusion matrix + classification report ─────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle(f'Classification Evaluation — {best_cls}', fontsize=13, fontweight='bold')

cm = confusion_matrix(y_test_cls, cls_preds[best_cls])
ConfusionMatrixDisplay(cm, display_labels=['No Drought','Drought']).plot(
    ax=axes[0], colorbar=False, cmap='Blues')
axes[0].set_title('Confusion Matrix')

# ROC comparison
from sklearn.metrics import roc_curve
for name, prob in cls_probs.items():
    fpr, tpr, _ = roc_curve(y_test_cls, prob)
    auc = roc_auc_score(y_test_cls, prob)
    axes[1].plot(fpr, tpr, label=f'{name} (AUC={auc:.3f})')
axes[1].plot([0,1],[0,1],'k--',linewidth=0.8,label='Random')
axes[1].set_xlabel('False Positive Rate'); axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curves'); axes[1].legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR/'classification_eval.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nClassification Report — {best_cls}:')
print(classification_report(y_test_cls, cls_preds[best_cls], target_names=['No Drought','Drought']))

---
## 8. Feature Importance & SHAP

In [ ]:
def plot_feature_importance(pipe, feature_names, title, top_n=25, save_path=None):
    model = pipe.named_steps['model']
    if not hasattr(model, 'feature_importances_'):
        return
    fi = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})
    fi = fi.sort_values('importance', ascending=False).head(top_n)
    colors = plt.cm.RdYlGn_r(np.linspace(0.1, 0.9, len(fi)))
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(fi['feature'][::-1], fi['importance'][::-1], color=colors[::-1])
    ax.set_title(title, fontweight='bold', fontsize=13)
    ax.set_xlabel('Importance')
    plt.tight_layout()
    if save_path: plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    return fi


plot_feature_importance(
    reg_pipes[best_reg], FEATURE_COLS,
    f'Feature Importance — {best_reg} (Regression)',
    save_path=FIGURES_DIR/'feature_importance_reg.png'
)

plot_feature_importance(
    cls_pipes[best_cls], FEATURE_COLS,
    f'Feature Importance — {best_cls} (Classification)',
    save_path=FIGURES_DIR/'feature_importance_cls.png'
)

In [ ]:
# ── SHAP values ───────────────────────────────────────────────
def compute_shap(pipe, X, feature_names, title, max_samples=500, save_path=None):
    try:
        X_t = pipe[:-1].transform(X.head(max_samples))
        explainer = shap.TreeExplainer(pipe.named_steps['model'])
        sv = explainer.shap_values(X_t)
        if isinstance(sv, list): sv = sv[1]
        plt.figure(figsize=(10, 8))
        shap.summary_plot(sv, X_t, feature_names=feature_names, show=False, max_display=20)
        plt.title(title, fontweight='bold', fontsize=13)
        plt.tight_layout()
        if save_path: plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.show()
    except Exception as e:
        logger.warning(f'SHAP failed: {e}')


compute_shap(
    reg_pipes[best_reg], X_test, FEATURE_COLS,
    f'SHAP Summary — {best_reg} (SPI-3 Regression)',
    save_path=FIGURES_DIR/'shap_regression.png'
)

compute_shap(
    cls_pipes[best_cls], X_test, FEATURE_COLS,
    f'SHAP Summary — {best_cls} (Drought Classification)',
    save_path=FIGURES_DIR/'shap_classification.png'
)

---
## 9. Forecast Visualization

In [ ]:
def plot_forecast(test_df, pipe, feature_cols, district, model_name, save_path=None):
    """Observed vs predicted SPI-3 time series for a single district."""
    d = test_df[test_df['district']==district].sort_values('date')
    y_obs  = d['target_spi3']
    y_pred = pipe.predict(d[feature_cols])
    dates  = d['date']

    fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
    fig.suptitle(
        f'3-Month Ahead Drought Forecast — {district} | {model_name}\n'
        f'Test period: {dates.min().date()} → {dates.max().date()}',
        fontsize=13, fontweight='bold'
    )

    ax = axes[0]
    ax.plot(dates, y_obs,  color='steelblue', lw=2, label='Observed SPI-3', zorder=3)
    ax.plot(dates, y_pred, color='orangered', lw=2, ls='--', label='Predicted SPI-3', zorder=3)
    ax.axhline(-1, color='red', ls=':', lw=1.5, label='Drought threshold')
    ax.axhline(0,  color='gray', ls='-', lw=0.5)
    ax.fill_between(dates, -4, 4, where=(y_obs<-1), alpha=0.15, color='red', label='Drought')
    ax.set_ylim(-3.5, 3.5); ax.set_ylabel('SPI-3'); ax.legend(fontsize=9)

    resid = y_obs.values - y_pred
    ax2 = axes[1]
    ax2.bar(dates, resid, color=np.where(resid>0,'steelblue','tomato'), alpha=0.75)
    ax2.axhline(0, color='black', lw=0.8)
    ax2.set_ylabel('Residual (Obs−Pred)')
    ax2.set_title(f'RMSE = {np.sqrt(mean_squared_error(y_obs, y_pred)):.3f}', fontsize=11)
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    plt.tight_layout()
    if save_path: plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()


for dist in ['Hargeisa', 'Erigavo', 'Berbera']:
    plot_forecast(
        test_df, reg_pipes[best_reg], FEATURE_COLS,
        dist, best_reg,
        save_path=FIGURES_DIR/f'forecast_{dist.lower()}.png'
    )

In [ ]:
# ── Correlation heatmap ───────────────────────────────────────
key_cols = ['rainfall_mm','spi_1','spi_3','temp_2m_c','soil_moisture',
            'pet_mm','asis_index','oni','rain_anom','sm_anom',
            'water_deficit','aridity_idx','target_spi3']
key_cols = [c for c in key_cols if c in master_df.columns]

fig, ax = plt.subplots(figsize=(13, 11))
corr = master_df[key_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=.5,
            ax=ax, annot_kws={'size': 8})
ax.set_title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES_DIR/'correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Seasonal analysis ─────────────────────────────────────────
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Somaliland Seasonal Climate Analysis', fontsize=14, fontweight='bold')
s_colors = {'Gu':'#27AE60','Deyr':'#2980B9','Hagaa':'#E67E22','Jilaal':'#E74C3C'}

# Seasonal rainfall boxplot
s_order = ['Gu','Deyr','Hagaa','Jilaal']
data_s  = [master_df[master_df['season']==s]['rainfall_mm'].dropna() for s in s_order]
bp = axes[0,0].boxplot(data_s, labels=s_order, patch_artist=True)
for patch, s in zip(bp['boxes'], s_order):
    patch.set_facecolor(s_colors[s]); patch.set_alpha(0.75)
axes[0,0].set_title('Rainfall by Season'); axes[0,0].set_ylabel('mm/month')

# Monthly climatology
mc = master_df.groupby('month')['rainfall_mm'].mean()
m_colors = ['#27AE60' if m in [4,5,6] else '#2980B9' if m in [10,11] else '#E74C3C' for m in range(1,13)]
axes[0,1].bar(range(1,13), mc.values, color=m_colors, edgecolor='white')
axes[0,1].set_xticks(range(1,13))
axes[0,1].set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
axes[0,1].set_title('Monthly Climatology'); axes[0,1].set_ylabel('Mean Rainfall (mm)')

# Drought frequency by district
df_by_d = (master_df.groupby('district').apply(lambda g: (g['spi_3']<-1).mean()*100)
           .sort_values())
c_d = plt.cm.RdYlGn_r(np.linspace(.1,.9,len(df_by_d)))
axes[1,0].barh(df_by_d.index, df_by_d.values, color=c_d)
axes[1,0].set_title('Drought Frequency by District'); axes[1,0].set_xlabel('%')

# Annual trend
ann = master_df.groupby('year').apply(lambda g: (g['spi_3']<-1).mean()*100)
axes[1,1].plot(ann.index, ann.values, color='tomato', lw=2, marker='o', ms=3)
axes[1,1].fill_between(ann.index, 0, ann.values, color='tomato', alpha=0.2)
z = np.polyfit(ann.index, ann.values, 1)
axes[1,1].plot(ann.index, np.poly1d(z)(ann.index), 'k--', lw=1.5, label=f'{z[0]:+.2f}%/yr')
axes[1,1].set_title('Annual Drought Trend'); axes[1,1].set_ylabel('Districts in Drought (%)'); axes[1,1].legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR/'seasonal_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Interactive drought map ───────────────────────────────────
drought_freq = (
    master_df.groupby('district')
    .apply(lambda g: (g['spi_3']<-1).mean()*100)
    .reset_index(name='drought_pct')
)
drought_freq['lon'] = drought_freq['district'].map(lambda d: DISTRICT_COORDS.get(d,(45,9.5))[0])
drought_freq['lat'] = drought_freq['district'].map(lambda d: DISTRICT_COORDS.get(d,(45,9.5))[1])

fig = px.scatter_mapbox(
    drought_freq, lat='lat', lon='lon',
    color='drought_pct', size='drought_pct',
    hover_name='district', color_continuous_scale='RdYlGn_r',
    size_max=45, zoom=6, center=dict(lat=9.8, lon=45.5),
    mapbox_style='open-street-map',
    title='Drought Frequency — Somaliland Districts (1981–2023)'
)
fig.update_layout(height=520)
fig.show()

---
## 10. Save Models & Data

In [ ]:
# Save all pipelines
for name, pipe in reg_pipes.items():
    p = MODELS_DIR / f'reg_{name.lower()}.pkl'
    joblib.dump(pipe, p, compress=3)
    print(f'Saved: {p.name} ({p.stat().st_size/1024:.0f} KB)')

for name, pipe in cls_pipes.items():
    p = MODELS_DIR / f'cls_{name.lower()}.pkl'
    joblib.dump(pipe, p, compress=3)
    print(f'Saved: {p.name} ({p.stat().st_size/1024:.0f} KB)')

# Save feature list
joblib.dump(FEATURE_COLS, MODELS_DIR / 'feature_columns.pkl')

# Save processed data
master_df.to_csv(PROC_DIR / 'features_full.csv', index=False)
train_df.to_csv(PROC_DIR / 'train.csv', index=False)
test_df.to_csv(PROC_DIR / 'test.csv', index=False)
reg_df.to_csv(PROC_DIR / 'regression_results.csv', index=False)
cls_df.to_csv(PROC_DIR / 'classification_results.csv', index=False)

print('\n✅ All models and data saved.')
print(f'   Models   → {MODELS_DIR}')
print(f'   Data     → {PROC_DIR}')
print(f'   Figures  → {FIGURES_DIR}')

---
## Summary

### Results

| Task | Best Model | Key Metric |
|------|-----------|------------|
| SPI-3 Regression | — | RMSE / R² |
| Drought Classification | — | AUC / F1 |

### Next Steps
- Replace synthetic data with real CHIRPS rasters + CDS API ERA5 download
- Add cross-validation using blocked time-series CV (`TimeSeriesSplit`)
- Tune hyperparameters with Optuna
- Deploy as a Streamlit dashboard (see `streamlit_app/`)
- Set up automated monthly retraining pipeline